# Simple Rasa NLP Example

## Step 1 

Import required Libraries 

In [ ]:
from rasa.nlu.training_data import load_data
from rasa.nlu import config
from rasa.nlu.model import Trainer
import logging
import os
import shutil


## Step 2 

Create data directory in project home if it does not exists or delete existing contents and re create 

In [ ]:
# create Data directory

current_home = os.getcwd()
if os.path.isdir(current_home+"/data") is False:
    os.mkdir(current_home+"/data")
else:
    shutil.rmtree(current_home+"/data")
    os.mkdir(current_home+"/data")

## Step 3 

Create training data and config file 

In [ ]:
# create Training data

training_data = """
## intent:greet
- hi there
- hello 
- how are you 

## intent:goodbye
- goodbye 
- bye 
- tc
- bye bye

## intent:identify_part
- part number is [AB3322](part_number)
- item number is [BN4345](part_number)
- its [NM3344](part_number)
- part is [IK7788](part_number)
- [BN5654](part_number) is the item number 
- its # [IK8787](part_number)
- the # is [89d89](part_number)
"""

config_data = """
language: "en"

pipeline: "supervised_embeddings"
"""

## Step 4 
Write Training data and config file back to disk 

In [ ]:
f = open(current_home+"/data/"+"nlu.md", "w")
f.write(training_data)
f.close()

c = open(current_home+"/data/"+"config.yml", "w")
c.write(config_data)
c.close()

## Step 5 
Train the RASA model with the training data 

In [ ]:
# Specify the Training files

training_data_file = current_home+"/data/"+"nlu.md"
config_file = current_home+"/data/"+"config.yml"

# Train the Model

training_data = load_data(training_data_file)
trainer = Trainer(config.load(config_file))
trainer.train(training_data)

## Step 6
Persist the RASA Model to disk 

In [ ]:
# Save the model
model_path = os.path.join("models", "current")
model_directory = trainer.persist(model_path, fixed_model_name="nlu")

print("Model trained. Stored in '{}'.".format(model_directory))


## Step 7 
Load the saved RASA Model for infrence 

In [ ]:
# Now lets test
from rasa.nlu.model import Interpreter
interp=Interpreter.load(model_dir=model_directory)

## Step 8

Use the model !!

In [ ]:
while True:
    print("Enter Text - ")
    message = input()
    output=interp.parse(message)
    print("______________________________________________________________________________________________________")
    print("\n")
    print("Predicted Intent - {}".format(output['intent']['name']))
    print("Predicted with confidence {}".format(output['intent']['confidence']))
    for entity in output['entities']:
        print("Extracted Entity name - {}  and its value {}".
              format(entity['entity'],entity['value']))
    print("\n")
    print("Raw json Response {}".format(output))